# UMAP Bench: Preprocess Results

This notebook flattens the benchmark JSON files into a single CSV with only fields that exist in the JSON.

In [1]:
import json
import glob
import os
import pandas as pd

results_dir = "results"
output_path = "preprocessed.csv"


In [4]:
rows = []
files = sorted(glob.glob(os.path.join(results_dir, "bench-runs-*.json")))

for path in files:
    with open(path, "r", encoding="utf-8") as f:
        data = json.load(f)

    top = {
        "generated_at": data.get("generatedAt"),
        "runs_declared": data.get("runs"),
        "wasm_features_file": data.get("wasmFeatures"),
        "wasm_preload": data.get("wasmPreload"),
    }

    machine = data.get("machine") or {}
    top.update({
        "machine_platform": machine.get("platform"),
        "machine_release": machine.get("release"),
        "machine_arch": machine.get("arch"),
        "cpu_model": machine.get("cpuModel"),
        "cpu_cores": machine.get("cpuCores"),
        "total_mem_bytes": machine.get("totalMemBytes"),
        "load_avg_1": (machine.get("loadAvg") or [None, None, None])[0],
        "load_avg_5": (machine.get("loadAvg") or [None, None, None])[1],
        "load_avg_15": (machine.get("loadAvg") or [None, None, None])[2],
        "hostname": machine.get("hostname"),
    })

    git = data.get("git") or {}
    top.update({
        "git_commit": git.get("commit"),
        "git_branch": git.get("branch"),
        "git_status_dirty": git.get("statusDirty"),
    })

    for result in data.get("results", []):
        base = dict(top)
        base.update({
            "result_run": result.get("run"),
            "result_wasm_features": result.get("wasmFeatures"),
            "result_success": result.get("success"),
            "result_duration_ms": result.get("durationMs"),
            "result_label": result.get("resultLabel"),
        })

        stats = result.get("stats") or {}
        base.update({
            "stats_start_time": stats.get("startTime"),
            "stats_duration_ms": stats.get("duration"),
            "stats_flaky": stats.get("flaky"),
        })

        for ui in result.get("uiMetrics") or []:
            for row in ui.get("rows") or []:
                out = dict(base)
                out.update({
                    "ui_run_id": row.get("runId"),
                    "ui_timestamp": row.get("timestamp"),
                    "scope": row.get("scope"),
                    "dataset_name": row.get("datasetName"),
                    "dataset_size": row.get("datasetSize"),
                    "dimensions": row.get("dimensions"),
                    "ui_wasm_features": row.get("wasmFeatures"),
                    "wasm_mode": row.get("wasmMode"),
                    "runtime_ms": row.get("runtimeMs"),
                    "memory_delta_mb": row.get("memoryDeltaMb"),
                    "trustworthiness": row.get("trustworthiness"),
                    "fps_avg": row.get("fpsAvg"),
                    "responsiveness_ms": row.get("responsivenessMs"),
                })
                rows.append(out)


In [5]:
columns = [
    "generated_at",
    "runs_declared",
    "result_run",
    "result_wasm_features",
    "result_success",
    "result_duration_ms",
    "result_label",
    "stats_start_time",
    "stats_duration_ms",
    "stats_flaky",
    "wasm_features_file",
    "wasm_preload",
    "machine_platform",
    "machine_release",
    "machine_arch",
    "cpu_model",
    "cpu_cores",
    "total_mem_bytes",
    "load_avg_1",
    "load_avg_5",
    "load_avg_15",
    "hostname",
    "git_commit",
    "git_branch",
    "git_status_dirty",
    "ui_run_id",
    "ui_timestamp",
    "scope",
    "dataset_name",
    "dataset_size",
    "dimensions",
    "ui_wasm_features",
    "wasm_mode",
    "runtime_ms",
    "memory_delta_mb",
    "trustworthiness",
    "fps_avg",
    "responsiveness_ms",
]

df = pd.DataFrame(rows).reindex(columns=columns)
df.to_csv(output_path, index=False)
df.head()


,generated_at,runs_declared,result_run,result_wasm_features,result_success,result_duration_ms,result_label,stats_start_time,stats_duration_ms,stats_flaky,...,dataset_name,dataset_size,dimensions,ui_wasm_features,wasm_mode,runtime_ms,memory_delta_mb,trustworthiness,fps_avg,responsiveness_ms
0,2026-01-31T12:39:27.635Z,10,1,none,True,43526,PASS,2026-01-31T12:39:45.906Z,25293.620,0,...,"Iris Dataset (150 points, 4D)",150,4,none,js,2326.9,-0.055005,0.986882,60.000000,8.600000
1,2026-01-31T12:39:27.635Z,10,1,none,True,43526,PASS,2026-01-31T12:39:45.906Z,25293.620,0,...,Small Random (80 points),80,10,none,js,3404.4,-4.720348,0.876677,48.204624,24.000000
2,2026-01-31T12:39:27.635Z,10,2,none,True,45124,PASS,2026-01-31T12:40:27.525Z,28781.219,0,...,"Iris Dataset (150 points, 4D)",150,4,none,js,2297.1,2.982440,0.991627,60.000000,9.072727
3,2026-01-31T12:39:27.635Z,10,2,none,True,45124,PASS,2026-01-31T12:40:27.525Z,28781.219,0,...,Small Random (80 points),80,10,none,js,3291.8,14.746518,0.830674,48.000000,22.900000
4,2026-01-31T12:39:27.635Z,10,3,none,True,46734,PASS,2026-01-31T12:41:11.846Z,31195.437,0,...,"Iris Dataset (150 points, 4D)",150,4,none,js,2334.0,1.219504,0.988568,59.994001,9.881818
